In [1]:
import os
import tensorflow as tf
import numpy as np

SEED = 1234
tf.random.set_seed(SEED)

In [2]:
# ImageDataGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Create generators to read images from dataset directory

# Get current working directory
cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'data')

# Batch size
bs = 16

# img shape
img_h = 256
img_w = 256

num_classes=3

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

Found 5052 images belonging to 3 classes.
Found 562 images belonging to 3 classes.


In [4]:
# Create Dataset objects

size = (256, 256)
# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
valid_dataset = valid_dataset.repeat()

In [5]:
# Load Xception network for transfer learning

xception = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [6]:
print(len(xception.layers))

132


In [7]:
# Create Model

finetuning = True

if finetuning:
    freeze_until = 10 # layer from which we want to fine-tune
    
    for layer in xception.layers[:freeze_until]:
        layer.trainable = False
else:
    xception.trainable = False
    
model = tf.keras.Sequential()
model.add(xception)
model.add(tf.keras.layers.Dense(units=512, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.AveragePooling2D(pool_size=(7,7)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
dense (Dense)                (None, 8, 8, 512)         1049088   
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 1, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0

0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

In [8]:
# Optimization params

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 5e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [9]:
callbacks = []

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
callbacks.append(es_callback)
reduce_rl_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3)
callbacks.append(reduce_rl_callback)

In [11]:
epochs = 50
model.fit(x=train_dataset, epochs=epochs,steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/50
316/316 [==============================] - 311s 985ms/step - loss: 1.1460 - accuracy: 0.6823 - val_loss: 1.8499 - val_accuracy: 0.6370
Epoch 2/50
316/316 [==============================] - 309s 979ms/step - loss: 0.5844 - accuracy: 0.8116 - val_loss: 0.8450 - val_accuracy: 0.6797
Epoch 3/50
316/316 [==============================] - 297s 939ms/step - loss: 0.4415 - accuracy: 0.8424 - val_loss: 0.3963 - val_accuracy: 0.8452
Epoch 4/50
316/316 [==============================] - 294s 929ms/step - loss: 0.3906 - accuracy: 0.8531 - val_loss: 0.3854 - val_accuracy: 0.8381
Epoch 5/50
316/316 [==============================] - 304s 961ms/step - loss: 0.3494 - accuracy: 0.8737 - val_loss: 0.3415 - val_accuracy: 0.8719
Epoch 6/50
316/316 [==============================] - 298s 941ms/step - loss: 0.3242 - accuracy: 0.8812 - val_loss: 0.4721 - val_accuracy: 0.7865
Epoch 7/50
316/316 [==============================] - 302s 956ms/step - loss: 0.2980 - accuracy: 0.8919 - val_loss: 0.4747 -

In [12]:
model.save('tl_saved_model_xception092')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tl_saved_model\assets


In [14]:
import pandas as pd
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results)

Found 450 validated image filenames belonging to 1 classes.
29/29 [==============================] - 10s 329ms/step
